In [0]:
# !pip install Faker names

In [0]:
import random
from datetime import datetime, timedelta
from faker import Faker
from pyspark.sql import SparkSession
from pyspark.sql.types import (StructType, StructField, IntegerType, StringType,
                               DateType)

# Inicializa o Faker para gerar dados em português do Brasil
fake = Faker('pt_BR')

# --- Configurações e Listas de Apoio ---

# Lista de comodidades que um hotel pode oferecer
COMODIDADES_DISPONIVEIS = [
    'Wi-Fi Grátis', 'Piscina', 'Academia', 'Spa', 'Estacionamento',
    'Restaurante', 'Bar', 'Serviço de Quarto', 'Recepção 24h',
    'Ar Condicionado', 'Pet Friendly', 'Salão de Eventos', 'Business Center'
]

# Listas para gerar nomes de hotéis mais realistas
PREFIXOS_NOMES = ['Grand', 'Royal', 'Plaza', 'Imperial', 'Golden', 'Paradise', 'Blue Tree']
SUFIXOS_NOMES = ['Hotel', 'Resort', 'Palace', 'Inn', 'Pousada', 'Tower']


def gerar_data_abertura():
    """Gera uma data de abertura aleatória entre 1990 e 2020."""
    data_inicio = datetime(1990, 1, 1)
    data_fim = datetime(2020, 12, 31)
    diferenca_dias = (data_fim - data_inicio).days
    dias_aleatorios = random.randint(0, diferenca_dias)
    return data_inicio + timedelta(days=dias_aleatorios)

def gerar_hoteis(total_hoteis=50):
    """
    Gera uma lista de dados fictícios de hotéis.

    Args:
        total_hoteis (int): O número total de hotéis a serem gerados.

    Returns:
        list: Uma lista de tuplas, onde cada tupla representa um hotel.
    """
    hoteis = []
    for i in range(total_hoteis):
        hotel_id = 101 + i
        
        # Gera um nome de hotel combinando partes aleatórias
        nome_hotel = f"{random.choice(PREFIXOS_NOMES)} {fake.city_suffix()} {random.choice(SUFIXOS_NOMES)}"
        
        cidade = fake.city()
        estado = fake.state_abbr()
        
        # Seleciona um número aleatório de comodidades da lista
        num_comodidades = random.randint(4, 10)
        comodidades_selecionadas = random.sample(COMODIDADES_DISPONIVEIS, num_comodidades)
        
        hotel = (
            hotel_id,
            nome_hotel,
            fake.street_address(),
            cidade,
            estado,
            fake.postcode(),
            'Brasil',
            random.randint(3, 5),  # Estrelas
            random.randint(50, 400), # Número de quartos
            ', '.join(comodidades_selecionadas), # Comodidades como string
            gerar_data_abertura(),
            fake.phone_number(),
            f"contato@{nome_hotel.lower().replace(' ', '')}.com",
            fake.name() # Nome do gerente
        )
        hoteis.append(hotel)
        
    return hoteis

# --- Execução do Script ---

# Em um ambiente como o Databricks, a sessão Spark 'spark' já vem iniciada.
# Caso contrário, você a inicializaria assim:
# spark = SparkSession.builder.appName("GeracaoHoteis").getOrCreate()

# Define o schema do DataFrame para garantir os tipos de dados corretos
schema_hoteis = StructType([
    StructField("hotel_id", IntegerType(), False),
    StructField("nome_hotel", StringType(), True),
    StructField("endereco", StringType(), True),
    StructField("cidade", StringType(), True),
    StructField("estado", StringType(), True),
    StructField("cep", StringType(), True),
    StructField("pais", StringType(), True),
    StructField("estrelas", IntegerType(), True),
    StructField("numero_quartos", IntegerType(), True),
    StructField("comodidades", StringType(), True),
    StructField("data_abertura", DateType(), True),
    StructField("telefone", StringType(), True),
    StructField("email", StringType(), True),
    StructField("gerente", StringType(), True)
])

# Gera a lista de dados dos hotéis
dados_hoteis = gerar_hoteis(150)

# Cria o DataFrame Spark a partir dos dados e schema definidos
df_hoteis = spark.createDataFrame(dados_hoteis, schema_hoteis)

# Mostra uma amostra dos dados gerados para verificação
print("Amostra dos dados de hotéis gerados:")
df_hoteis.show(10, truncate=False)

# Contagem de hotéis por estado para análise
print("\nContagem de hotéis por estado:")
df_hoteis.groupBy("estado").count().orderBy("count", ascending=False).show()

# --- Salvando os Dados ---

# Define o caminho de destino no catálogo do Databricks
catalog_path = "production.transient.pms_hoteis"

# Salva o DataFrame como uma tabela Delta, sobrescrevendo se já existir
df_hoteis.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable(catalog_path)

print(f"\nTabela de hotéis salva com sucesso em: {catalog_path}")
print(f"Total de registros gravados: {df_hoteis.count()}")

# Otimiza a tabela para melhorar o desempenho de futuras consultas
print("\nAplicando OPTIMIZE na tabela...")
spark.sql(f"""
OPTIMIZE {catalog_path}
ZORDER BY (estado, estrelas)
""")

print("Otimização concluída com sucesso!")

In [0]:
df = spark.sql("""
               select * from production.transient.erp_faturas
               """)
display(df.columns)

In [0]:
# -*- coding: utf-8 -*-
import random
from datetime import datetime, timedelta
from faker import Faker
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, split
from pyspark.sql.types import (StructType, StructField, IntegerType, StringType,
                               DoubleType, DateType)

# Inicializa o Faker para gerar dados em português do Brasil
fake = Faker('pt_BR')

# --- DADOS DE FORMAS DE PAGAMENTO ADICIONADOS ---
formas_pagamento_data = [
    ("Cartão de Crédito Visa", "Crédito"),
    ("Cartão de Crédito Mastercard", "Crédito"),
    ("Cartão de Crédito Amex", "Crédito"),
    ("Cartão de Débito Visa", "Débito"),
    ("Cartão de Débito Mastercard", "Débito"),
    ("Pix", "Transferência"),
    ("Dinheiro", "Dinheiro"),
    ("Faturado (Empresa)", "Outros")
]

# --- Dicionário de Serviços e Preços ---
SERVICOS_HOTEL = {
    'Restaurante': {
        'itens': [
            {'descricao': 'Café da manhã', 'valor_min': 25.00, 'valor_max': 45.00},
            {'descricao': 'Almoço executivo', 'valor_min': 45.00, 'valor_max': 75.00},
            # ... (demais itens)
        ]
    },
    # ... (demais serviços)
}

def gerar_data_consumo():
    """Gera uma data de consumo aleatória dos últimos 4 anos até hoje."""
    data_inicio = datetime.now() - timedelta(days=4*365)
    data_fim = datetime.now()
    diferenca_dias = (data_fim - data_inicio).days
    dias_aleatorios = random.randint(0, diferenca_dias)
    return data_inicio + timedelta(days=dias_aleatorios)

def gerar_consumo_servico(comodidades_hotel):
    """
    Gera um consumo de serviço aleatório.
    """
    servicos_disponiveis = [s for s in SERVICOS_HOTEL.keys() if s in comodidades_hotel]
    if not servicos_disponiveis:
        return None, None, None, None
    tipo_servico = random.choice(servicos_disponiveis)
    item = random.choice(SERVICOS_HOTEL[tipo_servico]['itens'])
    valor = round(random.uniform(item['valor_min'], item['valor_max']), 2)
    pago = random.choices(['Sim', 'Não'], weights=[85, 15])[0]
    return tipo_servico, item['descricao'], valor, pago

# --- FUNÇÃO DE GERAÇÃO ATUALIZADA ---
def gerar_consumos(hoteis_info, total_consumos=25000):
    """
    Gera uma lista de consumos dos hóspedes, associando cada consumo
    a um hotel existente e incluindo a forma de pagamento.
    """
    consumos = []
    hospede_ids = list(range(1, 8001))
    reserva_ids = list(range(10001, 50001))

    for i in range(total_consumos):
        hotel = random.choice(hoteis_info)
        tipo_servico, descricao, valor, pago = gerar_consumo_servico(hotel['comodidades'])
        
        # --- LÓGICA PARA PAGAMENTO ADICIONADA ---
        # Escolhe uma forma de pagamento aleatória da lista
        forma_pagamento, tipo_pagamento = random.choice(formas_pagamento_data)
        # --- FIM DA LÓGICA ADICIONADA ---

        if tipo_servico is None:
            continue
            
        consumo = (
            5001 + i,
            random.choice(reserva_ids),
            random.choice(hospede_ids),
            hotel['hotel_id'],
            random.randint(101, hotel['numero_quartos'] + 100),
            gerar_data_consumo(),
            tipo_servico,
            descricao,
            valor,
            pago,
            # --- NOVAS COLUNAS ADICIONADAS AO REGISTRO ---
            forma_pagamento,
            tipo_pagamento
        )
        consumos.append(consumo)
    
    return consumos

# --- Execução do Script ---

# 1. Carregar a tabela de hotéis
try:
    df_hoteis_base = spark.table("production.transient.pms_hoteis")
    hoteis_info = df_hoteis_base.select(
        col("hotel_id"),
        col("numero_quartos"),
        split(col("comodidades"), ", ").alias("comodidades") 
    ).collect()
    print(f"{len(hoteis_info)} hotéis carregados com sucesso.")
except Exception as e:
    print(f"Erro ao carregar a tabela 'hoteis': {e}")
    dbutils.notebook.exit("Tabela de hotéis não encontrada.")

# 2. Definir o schema do novo DataFrame
# --- SCHEMA ATUALIZADO ---
schema_consumos = StructType([
    StructField("consumo_id", IntegerType(), False),
    StructField("reserva_id", IntegerType(), True),
    StructField("hospede_id", IntegerType(), True),
    StructField("hotel_id", IntegerType(), True),
    StructField("quarto_id", IntegerType(), True),
    StructField("data_consumo", DateType(), True),
    StructField("tipo_servico", StringType(), True),
    StructField("descricao", StringType(), True),
    StructField("valor", DoubleType(), True),
    StructField("pago", StringType(), True),
    # --- NOVOS CAMPOS ADICIONADOS AO SCHEMA ---
    StructField("nome_forma_pagamento", StringType(), True),
    StructField("tipo_pagamento", StringType(), True)
])

# 3. Gerar os dados de consumo
dados_consumos = gerar_consumos(hoteis_info, 25000)

# 4. Criar o DataFrame Spark
df_consumos = spark.createDataFrame(dados_consumos, schema_consumos)

# 5. Análise e Verificação
print("\nAmostra dos dados de consumos gerados (com forma de pagamento):")
df_consumos.show(10, truncate=False)

print("\nTotal de consumos por tipo de serviço:")
df_consumos.groupBy("tipo_servico").count().orderBy(col("count").desc()).show()

# --- NOVA ANÁLISE DE VERIFICAÇÃO ---
print("\nTotal de consumos por tipo de pagamento:")
df_consumos.groupBy("tipo_pagamento").count().orderBy(col("count").desc()).show()

# --- Salvando os Dados ---
catalog_path = "production.transient.pdv_consumos"
df_consumos.write.format("delta").mode("overwrite").option("overwriteSchema", "true").saveAsTable(catalog_path)
print(f"\nTabela de consumos salva com sucesso em: {catalog_path}")

# Otimiza a tabela
spark.sql(f"OPTIMIZE {catalog_path} ZORDER BY (data_consumo, hotel_id, tipo_servico)")
print("Otimização concluída com sucesso!")

In [0]:
import random
from datetime import datetime, timedelta
from faker import Faker
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum, max, lit, rand, when
from pyspark.sql.types import (StructType, StructField, IntegerType, StringType,
                               DoubleType, DateType)

# Inicializa o Faker
fake = Faker('pt_BR')

# --- Lista de Apoio ATUALIZADA ---
FORMAS_PAGAMENTO_DATA = [
    ("Cartão de Crédito Visa", "Crédito"),
    ("Cartão de Crédito Mastercard", "Crédito"),
    ("Cartão de Crédito Amex", "Crédito"),
    ("Cartão de Débito Visa", "Débito"),
    ("Cartão de Débito Mastercard", "Débito"),
    ("Pix", "Transferência"),
    ("Dinheiro", "Dinheiro"),
    ("Faturado (Empresa)", "Outros")
]

def gerar_faturas_a_partir_de_consumos(df_consumos_agregados):
    """
    Gera os detalhes da fatura para cada reserva, incluindo o tipo de pagamento.
    """
    faturas = []
    reservas_para_faturar = df_consumos_agregados.collect()
    data_atual = datetime.now()

    for i, reserva in enumerate(reservas_para_faturar):
        fatura_id = 9001 + i
        reserva_id = reserva['reserva_id']
        hotel_id = reserva['hotel_id']
        subtotal_consumos = reserva['subtotal_consumos']
        ultima_data_consumo = reserva['ultima_data_consumo']

        valor_diarias = round(random.uniform(350.0, 4500.0), 2)
        valor_total = round(valor_diarias + subtotal_consumos, 2)
        impostos = round(valor_total * 0.12, 2)
        
        data_emissao = ultima_data_consumo + timedelta(days=1)
        data_vencimento = data_emissao + timedelta(days=random.randint(7, 20))
        
        if data_vencimento < data_atual.date():
            status_pgto = random.choices(['Pago', 'Em Atraso'], weights=[85, 15])[0]
        else:
            status_pgto = random.choices(['Pendente', 'Pago'], weights=[70, 30])[0]

        # --- LÓGICA DE PAGAMENTO ATUALIZADA ---
        forma_pgto = None
        tipo_pgto = None # Inicializa o tipo como nulo
        data_pagamento = None
        if status_pgto == 'Pago':
            # Define pesos para uma distribuição mais realista
            pesos_pagamento = [25, 25, 10, 10, 10, 15, 4, 1]
            # Desempacota o nome e o tipo da forma de pagamento
            forma_pgto, tipo_pgto = random.choices(FORMAS_PAGAMENTO_DATA, weights=pesos_pagamento, k=1)[0]
            
            dias_para_pagar = (min(data_vencimento, data_atual.date()) - data_emissao).days
            data_pagamento = data_emissao + timedelta(days=random.randint(0, dias_para_pagar)) if dias_para_pagar > 0 else data_emissao
        # --- FIM DA ATUALIZAÇÃO ---
        
        nota_fiscal = f"NFE-{reserva['estado_sigla']}-{hotel_id:03d}-{reserva_id:05d}"
        
        fatura = (
            fatura_id,
            reserva_id,
            hotel_id,
            data_emissao,
            data_vencimento,
            data_pagamento,
            valor_diarias,
            subtotal_consumos,
            impostos,
            valor_total,
            forma_pgto,
            tipo_pgto, # Coluna adicionada
            status_pgto,
            nota_fiscal
        )
        faturas.append(fatura)
        
    return faturas

# --- Execução do Script ---

# 1. Carregar as tabelas de dependência
try:
    df_consumos = spark.table("production.transient.pdv_consumos")
    df_hoteis = spark.table("production.transient.pms_hoteis")
    print("Tabelas 'consumos' e 'hoteis' carregadas com sucesso.")
except Exception as e:
    print(f"Erro ao carregar tabelas: {e}")
    dbutils.notebook.exit("Tabelas de dependência não encontradas.")

# 2. Agregar os consumos por reserva
df_consumos_agregados = df_consumos.groupBy("reserva_id", "hotel_id") \
    .agg(
        sum("valor").alias("subtotal_consumos"),
        max("data_consumo").alias("ultima_data_consumo")
    ) \
    .join(df_hoteis.select(col("hotel_id"), col("estado").alias("estado_sigla")), "hotel_id")

print(f"{df_consumos_agregados.count()} reservas únicas encontradas para faturamento.")

# 3. Definir o schema do DataFrame de faturas
# --- SCHEMA ATUALIZADO ---
schema_faturas = StructType([
    StructField("fatura_id", IntegerType(), False),
    StructField("reserva_id", IntegerType(), True),
    StructField("hotel_id", IntegerType(), True),
    StructField("data_emissao", DateType(), True),
    StructField("data_vencimento", DateType(), True),
    StructField("data_pagamento", DateType(), True),
    StructField("valor_diarias", DoubleType(), True),
    StructField("subtotal_consumos", DoubleType(), True),
    StructField("impostos", DoubleType(), True),
    StructField("valor_total", DoubleType(), True),
    StructField("forma_pagamento", StringType(), True),
    StructField("tipo_pagamento", StringType(), True), # Coluna adicionada
    StructField("status_pagamento", StringType(), True),
    StructField("nota_fiscal", StringType(), True)
])

# 4. Gerar os dados das faturas
dados_faturas = gerar_faturas_a_partir_de_consumos(df_consumos_agregados)

# 5. Criar o DataFrame Spark
df_faturas = spark.createDataFrame(dados_faturas, schema_faturas)

# 6. Análise e Verificação
print("\nAmostra das faturas geradas:")
df_faturas.select("fatura_id", "status_pagamento", "forma_pagamento", "tipo_pagamento").show(10, truncate=False)

print("\nDistribuição de faturas por tipo de pagamento:")
df_faturas.groupBy("tipo_pagamento").count().orderBy(col("count").desc()).show()

# --- Salvando os Dados ---
catalog_path = "production.transient.erp_faturas"
df_faturas.write.format("delta").mode("overwrite").option("overwriteSchema", "true").saveAsTable(catalog_path)
print(f"\nTabela de faturas salva com sucesso em: {catalog_path}")

# Otimiza a tabela
spark.sql(f"""
OPTIMIZE {catalog_path}
ZORDER BY (data_emissao, status_pagamento)
""")
print("Otimização concluída com sucesso!")

In [0]:
import random
from datetime import datetime, timedelta
from pyspark.sql import SparkSession
# Melhor Prática Definitiva: Importar functions com o alias F para evitar 100% dos conflitos de nome
import pyspark.sql.functions as F
from pyspark.sql.types import (StructType, StructField, IntegerType, StringType,
                               DateType)
from faker import Faker

# Configurar Faker para o Brasil
fake = Faker('pt_BR')

# --- (Suas funções e schemas permanecem os mesmos) ---

def gerar_hospedes(hospedes_a_criar):
    hospedes_completos = []
    TIPOS_CLIENTE = ['Recorrente', 'Novo', 'VIP', 'Premium', 'Corporativo']
    IDENTIDADES_GENERO = {"opcoes": ["Mulher Cis", "Homem Cis", "Mulher Trans", "Homem Trans", "Não-binário", "Prefiro não informar"], "pesos": [47, 46, 1, 1, 2, 3]}
    PRONOMES = {"opcoes": ["Ela/Dela", "Ele/Dele", "Elu/Delu", "Prefiro não informar"], "pesos": [48, 47, 2, 3]}
    ORIENTACOES_SEXUAIS = {"opcoes": ["Heterossexual", "Bissexual", "Homossexual", "Pansexual", "Assexual", "Prefiro não informar"], "pesos": [85, 4, 3, 2, 1, 5]}
    
    for hospede_info in hospedes_a_criar:
        hospede_id = hospede_info['hospede_id']
        data_ultima_estadia = hospede_info['data_ultima_estadia']
        ultimo_hotel_id = hospede_info['ultimo_hotel_id']
        sexo_nasc = random.choice(["Masculino", "Feminino"])
        identidade_genero = random.choices(IDENTIDADES_GENERO["opcoes"], weights=IDENTIDADES_GENERO["pesos"], k=1)[0]
        nome_registro = fake.name_male() if sexo_nasc == "Masculino" else fake.name_female()
        nome_social = nome_registro
        if ("Trans" in identidade_genero or "Não-binário" in identidade_genero) and random.random() < 0.9:
            nome_social = fake.name_female() if "Mulher" in identidade_genero else fake.name_male()
        hospede = (
            hospede_id, nome_registro, nome_social, fake.cpf(),
            fake.date_of_birth(minimum_age=18, maximum_age=85), sexo_nasc, identidade_genero,
            random.choices(PRONOMES["opcoes"], weights=PRONOMES["pesos"], k=1)[0],
            random.choices(ORIENTACOES_SEXUAIS["opcoes"], weights=ORIENTACOES_SEXUAIS["pesos"], k=1)[0],
            'Brasil', random.choice(TIPOS_CLIENTE), f"{nome_social.lower().replace(' ', '.')}@{random.choice(['gmail.com', 'hotmail.com'])}",
            fake.phone_number(), fake.state_abbr(), fake.city(), fake.postcode(),
            data_ultima_estadia, ultimo_hotel_id
        )
        hospedes_completos.append(hospede)
    return hospedes_completos

schema_hospedes = StructType([
    StructField("hospede_id", IntegerType(), False), StructField("nome_registro", StringType(), True),
    StructField("nome_social", StringType(), True), StructField("cpf", StringType(), True),
    StructField("data_nascimento", DateType(), True), StructField("sexo_atribuido_nascimento", StringType(), True),
    StructField("identidade_genero", StringType(), True), StructField("pronome", StringType(), True),
    StructField("orientacao_sexual", StringType(), True), StructField("nacionalidade", StringType(), True),
    StructField("tipo_cliente", StringType(), True), StructField("email", StringType(), True),
    StructField("telefone", StringType(), True), StructField("estado", StringType(), True),
    StructField("cidade", StringType(), True), StructField("cep", StringType(), True),
    StructField("data_ultima_estadia", DateType(), True), StructField("ultimo_hotel_id", IntegerType(), True)
])


# --- EXECUÇÃO DO SCRIPT ---
# Verifique se o nome da tabela de origem está correto
TABLE_NAME = "production.transient.pdv_consumos" 
try:
    df_consumos = spark.table(TABLE_NAME)
    print(f"Tabela '{TABLE_NAME}' carregada com sucesso.")
except Exception as e:
    print(f"Erro ao carregar a tabela de consumos: {e}")
    dbutils.notebook.exit("Tabela de consumos não encontrada.")

# Usando o alias F para garantir que as funções corretas do Spark sejam chamadas
df_hospedes_a_criar = df_consumos.groupBy("hospede_id") \
    .agg(
        F.max("data_consumo").alias("data_ultima_estadia"),
        F.first("hotel_id", ignorenulls=True).alias("ultimo_hotel_id")
    )

hospedes_info = df_hospedes_a_criar.collect()
print(f"Encontrados {len(hospedes_info)} hóspedes únicos para gerar perfis.")

print("Gerando perfis detalhados para cada hóspede...")
dados_hospedes = gerar_hospedes(hospedes_info)

df_hospedes = spark.createDataFrame(dados_hospedes, schema_hospedes)

print("\nAmostra dos dados de hóspedes gerados:")
df_hospedes.select("hospede_id", "nome_social", "data_ultima_estadia").show(5, truncate=False)

print("\nDistribuição de hóspedes por tipo de cliente:")
df_hospedes.groupBy("tipo_cliente").count().orderBy(F.col("count").desc()).show()

# --- Salvando os Dados ---
catalog_path = "production.transient.crm_hospedes"
df_hospedes.write.format("delta").mode("overwrite").option("overwriteSchema", "true").saveAsTable(catalog_path)
print(f"\nTabela de hóspedes salva com sucesso em: {catalog_path}")

spark.sql(f"OPTIMIZE {catalog_path} ZORDER BY (estado, tipo_cliente)")
print("Otimização concluída com sucesso!")

In [0]:
import random
from datetime import datetime, timedelta
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql.types import (StructType, StructField, IntegerType, StringType,
                               DoubleType, BooleanType)

# --- DOMÍNIOS DE DADOS ---

# Definição dos tipos de quarto e suas características base
TIPOS_QUARTO = {
    'Standard': {
        'capacidade': [1, 2], 'faixa_preco': [250.00, 400.00], 
        'descricao': 'Quarto confortável para estadias curtas.', 'percentual': 0.50
    },
    'Superior': {
        'capacidade': [2, 3], 'faixa_preco': [400.00, 650.00], 
        'descricao': 'Quarto espaçoso com vista e amenities adicionais.', 'percentual': 0.25
    },
    'Suíte Junior': {
        'capacidade': [2, 4], 'faixa_preco': [600.00, 900.00], 
        'descricao': 'Suíte com uma pequena área de estar integrada.', 'percentual': 0.15
    },
    'Suíte Executiva': {
        'capacidade': [2, 4], 'faixa_preco': [800.00, 1300.00], 
        'descricao': 'Suíte com sala de estar separada e acesso ao lounge executivo.', 'percentual': 0.07
    },
    'Suíte Presidencial': {
        'capacidade': [4, 6], 'faixa_preco': [1500.00, 3500.00], 
        'descricao': 'A acomodação mais luxuosa, com múltiplos ambientes.', 'percentual': 0.03
    }
}

# Status possíveis para um quarto
STATUS_QUARTO = ['Disponível', 'Ocupado', 'Em Limpeza', 'Em Manutenção', 'Bloqueado']

# --- FUNÇÃO PRINCIPAL ---

def gerar_quartos(hoteis_info, quartos_ocupados):
    """Gera a lista de todos os quartos para todos os hotéis."""
    quartos_gerados = []
    quarto_id_global = 1001 # Inicia um ID único global para os quartos

    for hotel in hoteis_info:
        total_quartos_hotel = hotel['numero_quartos']
        
        # Cria uma lista com os tipos de quarto a serem gerados para este hotel
        lista_tipos = []
        for tipo, config in TIPOS_QUARTO.items():
            num_quartos_tipo = int(total_quartos_hotel * config['percentual'])
            lista_tipos.extend([tipo] * num_quartos_tipo)
        
        # Ajusta para garantir que o total de quartos seja exato
        while len(lista_tipos) < total_quartos_hotel:
            lista_tipos.append('Standard') # Adiciona Standard para completar
        
        random.shuffle(lista_tipos)

        for i in range(total_quartos_hotel):
            tipo_quarto_atual = lista_tipos[i]
            config_quarto = TIPOS_QUARTO[tipo_quarto_atual]

            # Define o número do quarto (ex: 101, 102, 201, 202)
            andar = (i // 25) + 1 # Simula 25 quartos por andar
            numero_no_andar = (i % 25) + 1
            numero_quarto = f"{andar:02d}{numero_no_andar:02d}"

            # Define o status do quarto
            chave_ocupacao = (hotel['hotel_id'], int(numero_quarto))
            if chave_ocupacao in quartos_ocupados:
                status = random.choice(['Ocupado', 'Em Limpeza'])
            else:
                status = random.choices(
                    ['Disponível', 'Em Manutenção', 'Bloqueado'], 
                    weights=[95, 4, 1]
                )[0]
            
            # Calcula o preço base, influenciado pelas estrelas do hotel
            preco_base = random.uniform(config_quarto['faixa_preco'][0], config_quarto['faixa_preco'][1])
            preco_final = preco_base * (1 + (hotel['estrelas'] - 3) * 0.1) # +10% por estrela acima de 3

            quarto = (
                quarto_id_global,
                hotel['hotel_id'],
                int(numero_quarto),
                tipo_quarto_atual,
                config_quarto['descricao'],
                random.randint(config_quarto['capacidade'][0], config_quarto['capacidade'][1]),
                andar,
                status,
                round(preco_final, 2),
                random.choice([True, False]) # Simula se o quarto é para fumantes
            )
            quartos_gerados.append(quarto)
            quarto_id_global += 1
            
    return quartos_gerados

# --- SCHEMA DO DATAFRAME ---
schema_quartos = StructType([
    StructField("quarto_id", IntegerType(), False),
    StructField("hotel_id", IntegerType(), False),
    StructField("numero_quarto", IntegerType(), True),
    StructField("tipo_quarto", StringType(), True),
    StructField("descricao", StringType(), True),
    StructField("capacidade_maxima", IntegerType(), True),
    StructField("andar", IntegerType(), True),
    StructField("status_atual", StringType(), True),
    StructField("preco_diaria_base", DoubleType(), True),
    StructField("permite_fumantes", BooleanType(), True)
])

# --- EXECUÇÃO DO SCRIPT ---

# 1. Carregar as tabelas de dependência
try:
    df_hoteis = spark.table("production.transient.pms_hoteis")
    df_consumos = spark.table("production.transient.pdv_consumos")
    print("Tabelas 'hoteis' e 'consumos' carregadas com sucesso.")
except Exception as e:
    print(f"Erro ao carregar tabelas de dependência: {e}")
    dbutils.notebook.exit("Não foi possível carregar as tabelas base.")

# 2. Coletar informações dos hotéis para a geração
hoteis_info = df_hoteis.select(
    "hotel_id", "numero_quartos", "estrelas"
).collect()

# 3. Identificar quartos com atividade recente para definir o status como 'Ocupado'
data_limite = datetime.now() - timedelta(days=2)
df_quartos_ocupados = df_consumos.filter(col("data_consumo") >= data_limite) \
    .select("hotel_id", "quarto_id").distinct()

# Cria um conjunto (set) para busca rápida e eficiente
quartos_ocupados_set = set(
    (row.hotel_id, row.quarto_id) for row in df_quartos_ocupados.collect()
)
print(f"Encontrados {len(quartos_ocupados_set)} quartos com atividade recente.")

# 4. Gerar os dados dos quartos
print("Gerando o inventário de quartos para cada hotel...")
dados_quartos = gerar_quartos(hoteis_info, quartos_ocupados_set)

# 5. Criar o DataFrame Spark
df_quartos = spark.createDataFrame(dados_quartos, schema_quartos)

# 6. Análise e Verificação
print("\nAmostra dos quartos gerados:")
df_quartos.show(10, truncate=False)

print("\nContagem de quartos por tipo e hotel (Top 10 combinações):")
df_quartos.groupBy("hotel_id", "tipo_quarto").count().orderBy("count", ascending=False).show(10)

print("\nDistribuição de status dos quartos:")
df_quartos.groupBy("status_atual").count().orderBy("count", ascending=False).show()

# --- Salvando os Dados ---
catalog_path = "production.transient.pms_quartos"

df_quartos.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable(catalog_path)

print(f"\nTabela de quartos salva com sucesso em: {catalog_path}")
print(f"Total de registros gravados: {df_quartos.count()}")

# Otimiza a tabela para melhorar o desempenho
print("\nAplicando OPTIMIZE na tabela...")
spark.sql(f"""
OPTIMIZE {catalog_path}
ZORDER BY (hotel_id, tipo_quarto, status_atual)
""")
print("Otimização concluída com sucesso!")

In [0]:
import random
from datetime import datetime, timedelta
from pyspark.sql import SparkSession
# Importações de funções ajustadas para incluir as de agregação
from pyspark.sql.functions import col, min as spark_min, first, count, avg 
from pyspark.sql.types import (StructType, StructField, IntegerType, StringType,
                               DoubleType, DateType)

# --- DOMÍNIOS DE DADOS ---

# OTAs (Online Travel Agencies) com suas comissões e prefixos de ID
OTAS = [
    {'nome': 'Booking.com', 'comissao_min': 12.0, 'comissao_max': 18.0, 'prefixo': 'BK'},
    {'nome': 'Expedia', 'comissao_min': 15.0, 'comissao_max': 20.0, 'prefixo': 'EX'},
    {'nome': 'Decolar', 'comissao_min': 12.0, 'comissao_max': 16.0, 'prefixo': 'DE'},
    {'nome': 'Hoteis.com', 'comissao_min': 14.0, 'comissao_max': 19.0, 'prefixo': 'HO'},
    {'nome': 'CVC', 'comissao_min': 8.0, 'comissao_max': 12.0, 'prefixo': 'CV'}
]

# Canais de reserva direta (sem comissão)
CANAIS_DIRETOS = ['Website Hotel', 'Telefone', 'Balcão', 'E-mail Direto']

# --- FUNÇÕES AUXILIARES ---

def calcular_ajuste_comissao(tipo_quarto):
    """Retorna um percentual de ajuste de comissão baseado no luxo do quarto."""
    ajustes = {
        'Standard': 0.0,
        'Superior': 0.5,
        'Suíte Junior': 1.5,
        'Suíte Executiva': 2.5,
        'Suíte Presidencial': 4.0
    }
    # Retorna 0.0 se o tipo de quarto for None ou não estiver no dicionário
    return ajustes.get(tipo_quarto, 0.0) if tipo_quarto else 0.0

# --- FUNÇÃO PRINCIPAL ---

def gerar_canais_de_reserva(reservas_info):
    """
    Gera os dados do canal de origem para uma lista de reservas existentes.
    """
    dados_canal = []
    
    lista_fontes = ['OTA', 'Direto']
    pesos_fontes = [75, 25]

    for reserva in reservas_info:
        reserva_id = reserva['reserva_id']
        tipo_quarto = reserva['tipo_quarto']
        data_checkin = reserva['data_primeiro_consumo']

        fonte_reserva = random.choices(lista_fontes, weights=pesos_fontes)[0]
        
        data_reserva = data_checkin - timedelta(days=random.randint(1, 90))

        if fonte_reserva == 'OTA':
            ota = random.choice(OTAS)
            canal = ota['nome']
            reserva_canal_id = f"{ota['prefixo']}-{reserva_id}"
            
            comissao_base = random.uniform(ota['comissao_min'], ota['comissao_max'])
            ajuste_quarto = calcular_ajuste_comissao(tipo_quarto)
            comissao_final = round(comissao_base + ajuste_quarto, 2)
            
        else: # Canal Direto
            canal = random.choice(CANAIS_DIRETOS)
            reserva_canal_id = f"DIR-{reserva_id}"
            comissao_final = 0.0

        registro_canal = (
            reserva_canal_id,
            reserva_id,
            canal,
            fonte_reserva,
            comissao_final,
            data_reserva
        )
        dados_canal.append(registro_canal)
        
    return dados_canal

# --- SCHEMA DO DATAFRAME ---
schema_reservas_canal = StructType([
    StructField("reserva_canal_id", StringType(), False),
    StructField("reserva_id", IntegerType(), False),
    StructField("canal_reserva", StringType(), True),
    StructField("fonte_reserva", StringType(), True), 
    StructField("percentual_comissao", DoubleType(), True),
    StructField("data_reserva", DateType(), True)
])

# --- EXECUÇÃO DO SCRIPT ---

# 1. Carregar as tabelas de dependência
try:
    df_consumos = spark.table("production.transient.pdv_consumos")
    df_quartos = spark.table("production.transient.pms_quartos")
    print("Tabelas 'consumos' e 'quartos' carregadas com sucesso.")
except Exception as e:
    print(f"Erro ao carregar tabelas de dependência: {e}")
    dbutils.notebook.exit("Não foi possível carregar as tabelas base.")

# 2. Identificar cada reserva única e buscar as informações necessárias
df_reservas_base = df_consumos.groupBy("reserva_id") \
    .agg(
        first("hotel_id", ignorenulls=True).alias("hotel_id"),
        first("quarto_id", ignorenulls=True).alias("quarto_id"),
        spark_min("data_consumo").alias("data_primeiro_consumo")
    )

# 3. Juntar com a tabela de quartos para obter o tipo de quarto
df_reservas_com_quarto = df_reservas_base.join(
    df_quartos.select("hotel_id", col("numero_quarto").alias("quarto_id"), "tipo_quarto"),
    on=["hotel_id", "quarto_id"],
    how="left"
).select("reserva_id", "tipo_quarto", "data_primeiro_consumo") # Seleciona apenas as colunas necessárias

reservas_info = df_reservas_com_quarto.collect()
print(f"Encontradas {len(reservas_info)} reservas únicas para processar.")

# 4. Gerar os dados do canal para cada reserva
print("Gerando dados do canal de origem para cada reserva...")
dados_finais = gerar_canais_de_reserva(reservas_info)

# 5. Criar o DataFrame Spark
df_reservas_canal = spark.createDataFrame(dados_finais, schema_reservas_canal)

# 6. Análise e Verificação
print("\nAmostra dos dados gerados:")
df_reservas_canal.show(10, truncate=False)

print("\nDistribuição de reservas por fonte:")
df_reservas_canal.groupBy("fonte_reserva").count().orderBy(col("count").desc()).show()

# =======================================================================
# CORREÇÃO APLICADA AQUI
# =======================================================================
print("\nComissão média e total de reservas por canal:")
df_reservas_canal.groupBy("canal_reserva") \
    .agg(
        count("*").alias("total_reservas"),
        avg("percentual_comissao").alias("comissao_media")
    ) \
    .orderBy(col("total_reservas").desc()) \
    .show()
# =======================================================================

# --- Salvando os Dados ---
catalog_path = "production.transient.crs_reservas_canal"

df_reservas_canal.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable(catalog_path)

print(f"\nTabela de canais de reserva salva com sucesso em: {catalog_path}")
print(f"Total de registros gravados: {df_reservas_canal.count()}")

# Otimiza a tabela para melhorar o desempenho
print("\nAplicando OPTIMIZE na tabela...")
spark.sql(f"""
OPTIMIZE {catalog_path}
ZORDER BY (data_reserva, fonte_reserva)
""")
print("Otimização concluída com sucesso!")

In [0]:
import random
from datetime import datetime
from pyspark.sql import SparkSession
from pyspark.sql.functions import (col, min as spark_min, max as spark_max, first, 
                                   datediff, when, lit, rand)
from pyspark.sql.types import (StructType, StructField, IntegerType, StringType,
                               DoubleType, DateType)

# --- DOMÍNIOS DE DADOS ---

# Lista de observações comuns para adicionar variedade aos dados
OBSERVACOES_COMUNS = [
    'Solicitou andar alto', 'Preferência por quarto silencioso', 'Cliente VIP', 
    'Comemoração de aniversário', 'Lua de mel', 'Check-in antecipado solicitado',
    'Check-out tardio aprovado', 'Necessita de berço', 'Alérgico a penas',
    'Pagamento será feito no check-in', None # Para simular reservas sem observações
]

# --- FUNÇÃO PRINCIPAL ---

def criar_reservas_consolidadas(spark):
    """
    Consolida dados de várias tabelas para criar a tabela mestre de reservas.
    """
    
    # 1. Carregar todas as tabelas de dependência
    try:
        df_consumos = spark.table("production.transient.pdv_consumos")
        df_quartos = spark.table("production.transient.pms_quartos")
        df_canais = spark.table("production.transient.crs_reservas_canal")
        df_hospedes = spark.table("production.transient.crm_hospedes")
        print("Tabelas de dependência carregadas com sucesso.")
    except Exception as e:
        print(f"Erro ao carregar tabelas de dependência: {e}")
        dbutils.notebook.exit("Não foi possível carregar as tabelas base.")

    # 2. Agregar os consumos para definir o período da estadia de cada reserva
    df_estadias = df_consumos.groupBy("reserva_id") \
        .agg(
            first("hospede_id").alias("hospede_id"),
            first("hotel_id").alias("hotel_id"),
            first("quarto_id").alias("numero_quarto"),
            spark_min("data_consumo").alias("data_checkin"),
            spark_max("data_consumo").alias("data_checkout")
        )

    # 3. Juntar as informações para enriquecer os dados da reserva
    
    # Adiciona dados do canal de origem (data da reserva, nome do canal)
    df_com_canal = df_estadias.join(
        df_canais.select("reserva_id", "data_reserva", "canal_reserva"),
        "reserva_id",
        "inner" # Usamos inner join para garantir que toda reserva tenha um canal
    )

    # Adiciona dados do quarto (preço da diária)
    df_com_quarto = df_com_canal.join(
        df_quartos.select("hotel_id", "numero_quarto", "preco_diaria_base"),
        ["hotel_id", "numero_quarto"],
        "left" # Usamos left join caso algum quarto não seja encontrado
    )

    # 4. Calcular campos derivados (noites, valor, status)
    DATA_ATUAL = datetime.strptime("2025-10-04", "%Y-%m-%d").date()

    df_final = df_com_quarto \
        .withColumn("numero_noites", datediff(col("data_checkout"), col("data_checkin")) + 1) \
        .withColumn("valor_total_estadia", 
                    (col("numero_noites") * col("preco_diaria_base")).cast(DoubleType())) \
        .withColumn("status_reserva", 
            when(col("data_checkout") < DATA_ATUAL, "Finalizada")
            .when((col("data_checkin") <= DATA_ATUAL) & (col("data_checkout") >= DATA_ATUAL), "Hospedado")
            .otherwise("Confirmada")
        )

    # 5. Adicionar observações aleatórias
    # Gerar a lista de observações como uma coluna literal do Spark
    observacoes_list = spark.createDataFrame([(o,) for o in OBSERVACOES_COMUNS], ["observacoes"])
    # Adicionar um ID aleatório para fazer o join
    df_final = df_final.withColumn("rand_join_key", (rand() * len(OBSERVACOES_COMUNS)).cast("int"))
    observacoes_list = observacoes_list.withColumn("rand_join_key", (rand() * len(OBSERVACOES_COMUNS)).cast("int"))
    
    # Selecionar e renomear as colunas para o formato final
    df_reservas = df_final.select(
        col("reserva_id"),
        col("hospede_id"),
        col("numero_quarto").alias("quarto_id"),
        col("hotel_id"),
        col("data_reserva"),
        col("data_checkin"),
        col("data_checkout"),
        col("numero_noites"),
        col("canal_reserva"),
        col("status_reserva").alias("status"),
        col("valor_total_estadia").alias("valor_total"),
        lit(None).cast(StringType()).alias("observacoes") # Adicionado como placeholder
    )
    
    return df_reservas

# --- Schema Final ---
# O schema é inferido diretamente da transformação do Spark
    
# --- EXECUÇÃO DO SCRIPT ---

spark = SparkSession.builder.appName("GeracaoReservas").getOrCreate()

print("Iniciando a consolidação da tabela de reservas...")
df_reservas = criar_reservas_consolidadas(spark)

# --- Análise e Verificação ---
print(f"\nTotal de reservas consolidadas: {df_reservas.count()}")
print("\nAmostra dos dados finais:")
df_reservas.show(10, truncate=False)

print("\nDistribuição de reservas por status:")
df_reservas.groupBy("status").count().orderBy("count", ascending=False).show()

print("\nValor total de reservas por canal:")
df_reservas.groupBy("canal_reserva") \
    .agg({"valor_total": "sum", "reserva_id": "count"}) \
    .withColumnRenamed("sum(valor_total)", "faturamento_total") \
    .withColumnRenamed("count(reserva_id)", "numero_de_reservas") \
    .orderBy("faturamento_total", ascending=False) \
    .show()

# --- Salvando os Dados ---
catalog_path = "production.transient.pms_reservas"

df_reservas.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable(catalog_path)

print(f"\nTabela de reservas salva com sucesso em: {catalog_path}")

# Otimiza a tabela para melhorar o desempenho
print("\nAplicando OPTIMIZE na tabela...")
spark.sql(f"""
OPTIMIZE {catalog_path}
ZORDER BY (data_checkin, hotel_id)
""")
print("Otimização concluída com sucesso!")
